In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [8]:
df = df.drop(['id'], axis=1)

In [9]:
df

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [10]:
X = df.drop(['label'], axis=1)

In [11]:
y = df['label']

In [12]:
# X.head()

In [13]:
# y.head()

In [16]:
X.shape

(18285, 3)

In [17]:
y.shape

(18285,)

In [18]:
vocab_size = 5000

In [19]:
messages = X.copy()

In [21]:
# messages

In [22]:
messages.reset_index(inplace=True)

In [24]:
# messages

In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Darpan
[nltk_data]     Chanana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# DATA PREPROCESSING

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
# corpus

In [30]:
one_hot_values = [one_hot(word, vocab_size) for word in corpus]

In [32]:
# one_hot_values

In [34]:
# EMBDEDDING REPRESENTATION

embedded_docs = pad_sequences(one_hot_values, padding='pre', maxlen=20)
print(embedded_docs)

[[   0    0    0 ... 2575 4854 1875]
 [   0    0    0 ... 4676 4256  754]
 [   0    0    0 ...  135 3674 1481]
 ...
 [   0    0    0 ... 3859 3631  605]
 [   0    0    0 ...  493 2562 3153]
 [   0    0    0 ... 3585 3309 2878]]


In [38]:
# embedded_docs[5]
# embedded_docs[450]
embedded_docs[2000]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1548, 1288, 1648, 3112,   51, 1438, 1648])

In [40]:
sentence_length = 20
embedding_vector_size = 50

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_size, input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [51]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [53]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [54]:
X_final.shape

(18285, 20)

In [55]:
y_final.shape

(18285,)

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=101)

In [58]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 9s 29ms/step - loss: 0.2977 - accuracy: 0.8640
Epoch 2/10
229/229 [==============================] - 6s 28ms/step - loss: 0.1355 - accuracy: 0.9467
Epoch 3/10
229/229 [==============================] - 7s 29ms/step - loss: 0.1004 - accuracy: 0.9637
Epoch 4/10
229/229 [==============================] - 6s 28ms/step - loss: 0.0721 - accuracy: 0.9746
Epoch 5/10
229/229 [==============================] - 6s 28ms/step - loss: 0.0467 - accuracy: 0.9833
Epoch 6/10
229/229 [==============================] - 6s 27ms/step - loss: 0.0351 - accuracy: 0.9888
Epoch 7/10
229/229 [==============================] - 6s 26ms/step - loss: 0.0220 - accuracy: 0.9926
Epoch 8/10
229/229 [==============================] - 6s 24ms/step - loss: 0.0133 - accuracy: 0.9962
Epoch 9/10
229/229 [==============================] - 6s 27ms/step - loss: 0.0066 - accuracy: 0.9986
Epoch 10/10
229/229 [==============================] - 6s 26ms/step - loss: 0.0050 - accura

In [62]:
model.metrics_names

['loss', 'accuracy']

In [63]:
model.evaluate(X_test, y_test)

115/115 [==============================] - 1s 12ms/step - loss: 0.5839 - accuracy: 0.9130


[0.583911657333374, 0.9130434989929199]